### Data from bash:

| Key         | Value       | Command     |
| ----------- | ----------- | ----------- |
| Number of files in clips | 2130366 | `ls -f \| wc -l` (-2) |
| Paragraph   | Text        | Blah |

In [2]:
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
ROOT = '_cv_corpus/en/'
files = ['validated', 'invalidated', 'other']

dfs = {fname: pd.read_csv(f'{ROOT}{fname}.tsv', sep='\t', header=0) for fname in files}

df = pd.concat(dfs, names=['validity']).reset_index(level=0)


/var/folders/n9/c_vk64v10sxbms738l3v_sk00000gp/T/ipykernel_2318/162273917.py:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = {fname: pd.read_csv(f'{ROOT}{fname}.tsv', sep='\t', header=0) for fname in files}
/var/folders/n9/c_vk64v10sxbms738l3v_sk00000gp/T/ipykernel_2318/162273917.py:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = {fname: pd.read_csv(f'{ROOT}{fname}.tsv', sep='\t', header=0) for fname in files}
/var/folders/n9/c_vk64v10sxbms738l3v_sk00000gp/T/ipykernel_2318/162273917.py:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = {fname: pd.read_csv(f'{ROOT}{fname}.tsv', sep='\t', header=0) for fname in files}


In [4]:
df.head()

# print()


,validity,client_id,path,sentence,up_votes,down_votes,age,gender,accents,locale,segment
0,validated,000abb3006b78ea4c1144e55d9d158f05a9db011016051...,common_voice_en_27710027.mp3,"Joe Keaton disapproved of films, and Buster al...",3,1,NaN,NaN,NaN,en,NaN
1,validated,0013037a1d45cc33460806cc3f8ecee9d536c45639ba4c...,common_voice_en_699711.mp3,She'll be all right.,2,1,NaN,NaN,NaN,en,NaN
2,validated,0014c5a3e5715a54855257779b89c2bb498d470b225866...,common_voice_en_21953345.mp3,six,3,2,NaN,NaN,NaN,en,Benchmark
3,validated,001509f4624a7dee75247f6a8b642c4a0d09f8be3eeea6...,common_voice_en_18132047.mp3,All's well that ends well.,2,0,NaN,NaN,NaN,en,NaN
4,validated,001519f234e04528a2b36158c205dbe61c8da45ab0242f...,common_voice_en_27340672.mp3,It is a busy market town that serves a large s...,2,0,NaN,NaN,NaN,en,NaN


In [5]:
durations = pd.read_csv('_cv_corpus/en/durations.csv', header=None, names=['duration', 'filename'])
print(durations.head())
def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    return text

durations_dict = {filename[7:]: duration for duration, filename in durations.values}
for filename, duration in durations_dict.items():
    print(filename, duration)
    break


   duration                          filename
0     2.232       clips/common_voice_en_1.mp3
1     2.184      clips/common_voice_en_10.mp3
2     2.976     clips/common_voice_en_100.mp3
3    11.760    clips/common_voice_en_1000.mp3
4     4.704   clips/common_voice_en_10000.mp3
common_voice_en_1.mp3 2.232


In [6]:
from pprint import pprint

metadata = {
    "gender":{},
    "accents":{},
}

for index, row in tqdm(df.iterrows(), total=len(df.index)):
    for col in ["gender", "accents"]:
        if row[col] not in metadata[col]:
            metadata[col][row[col]] = {fname: 
                {
                    "duration": 0.0,
                    "count": 0
                } for fname in files
            }
        metadata[col][row[col]][row["validity"]]["count"] += 1
        metadata[col][row[col]][row["validity"]]["duration"] += durations_dict[row["path"]]
    
pprint(metadata)

  0%|          | 0/2129956 [00:00<?, ?it/s]

{'accents': {nan: {'invalidated': {'count': 147950,
                                   'duration': 886668.8467770012},
                   'other': {'count': 138560, 'duration': 670168.5233890101},
                   'validated': {'count': 746937,
                                 'duration': 3789530.167449975}},
             '2nd Language ': {'invalidated': {'count': 4, 'duration': 19.872},
                               'other': {'count': 15, 'duration': 74.232},
                               'validated': {'count': 16,
                                             'duration': 74.55600000000001}},
             'A variety of Texan English with some German influence that has undergone the cot-caught merger': {'invalidated': {'count': 1,
                                                                                                                                'duration': 3.168},
                                                                                                            

In [17]:
total_gender_duration_hours = {
    'male': 0,
    'female': 0,
}
for gender in total_gender_duration_hours.keys():
    for validity, data in metadata['gender'][gender].items():
        total_gender_duration_hours[gender] += data["duration"]

total_gender_duration_hours = {k:v/60/60 for k,v in total_gender_duration_hours.items()}
pprint(total_gender_duration_hours)
validated_gender_duration_percentage = {
    'male': 0,
    'female': 0,
}
for gender in total_gender_duration_hours.keys():
    validated_gender_duration_percentage[gender] = (metadata['gender'][gender]['validated']['duration']/60/60) / total_gender_duration_hours[gender]
pprint(validated_gender_duration_percentage)

{'female': 500.8529786399982, 'male': 1335.2087957998654}
{'female': 0.7470712897151858, 'male': 0.7694266625503334}


In [20]:
import numpy as np
total_accented_duration = 0
total_validated_accent_duration = 0
total_duration = 0
for accent in metadata['accents']:
    for validity, data in metadata['accents'][accent].items():
        total_duration += data["duration"]
        if accent is not np.nan:
            total_accented_duration += data["duration"]
            if validity == 'validated':
                total_validated_accent_duration += data["duration"]

print(f'Total duration: {total_duration/60/60} hours')
print(f'Total accented duration: {total_accented_duration/60/60} hours')
print(f'Total validated accented duration: {total_validated_accent_duration/60/60} hours')
print(f'Total accented duration percentage: {total_accented_duration/total_duration}')
print(f'Total validated accented duration percentage: {total_validated_accent_duration/total_accented_duration}')

Total duration: 3049.793850644436 hours
Total accented duration: 1564.6917568622237 hours
Total validated accented duration: 1187.506949748334 hours
Total accented duration percentage: 0.5130483676893757
Total validated accented duration percentage: 0.7589398643792419


In [40]:
pprint([k for k,v in metadata["accents"].items()])

[nan,
 'England English,United States English',
 'Hong Kong English',
 'United States English,American Midwest',
 'England English',
 'United States English',
 'United States English,wolof',
 'Australian English',
 'Southern African (South Africa, Zimbabwe, Namibia)',
 'India and South Asia (India, Pakistan, Sri Lanka)',
 'United States English,midwestern US',
 'Australian English,England English,Japanese',
 'Canadian English',
 'Scottish English',
 'Filipino',
 "A'lo",
 'Swedish English',
 'England English,Southern England',
 'United States English,i have some pronunciation issues because of oral '
 'surgery and a hidden southern accent',
 'Finnish',
 'United States English,Spoke Chinese when little,90% Pennsylvanian accent, '
 '10% Chinese accent',
 'Singaporean English',
 'West Indies and Bermuda (Bahamas, Bermuda, Jamaica, Trinidad)',
 'Spanish bilingual',
 'Irish English',
 'United States English,England English',
 'United States English,Silicon Valley Native',
 'Not bad',
 'Unite